In [1]:
DF_PATH = "../data/processed/01_preprocessed_df.pkl"
# ROLES_PATH = "../data/raw/roles_short_names.csv" 
FIG_DIR = "../reports/figures"

ROLE_COLS      = ['DevType']
TECH_COLS      = ['LanguageHaveWorkedWith',
                  'DatabaseHaveWorkedWith',
                  'PlatformHaveWorkedWith',
                  'WebframeHaveWorkedWith',
                  'MiscTechHaveWorkedWith',
                  'ToolsTechHaveWorkedWith',
                  'NEWCollabToolsHaveWorkedWith']

In [105]:
# Load packages
import pandas as pd 
import numpy as np
import logging
import pickle
import os

import plotly 
import plotly.graph_objects as go
# from plotly.subplots import make_subplots
import plotly.express as px
import plotly.figure_factory as ff

from sklearn.manifold import TSNE
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler

from matplotlib import pyplot as plt
# from scipy.cluster.hierarchy import dendrogram

_________

## Read data and preprocess

In [3]:
# Read Data 
df = pd.read_pickle(DF_PATH)
# roles_names = pd.read_csv(ROLES_PATH, sep=';')

In [4]:
df.head()

,Employment,RemoteWork,EdLevel,YearsCode,YearsCodePro,DevType,Country,CompTotal,LanguageHaveWorkedWith,LanguageWantToWorkWith,...,ToolsTechHaveWorkedWith,ToolsTechWantToWorkWith,NEWCollabToolsHaveWorkedWith,NEWCollabToolsWantToWorkWith,OpSysProfessional use,OpSysPersonal use,VersionControlSystem,VCInteraction,Gender,WorkExp
0,[],NaN,NaN,NaN,NaN,[],NaN,NaN,[],[],...,[],[],[],[],[],[],[],[],[],NaN
1,"[Employed, full-time]",Fully remote,NaN,NaN,NaN,[],Canada,NaN,"[JavaScript, TypeScript]","[Rust, TypeScript]",...,[],[],[],[],[macOS],[Windows Subsystem for Linux (WSL)],[Git],[],[],NaN
2,"[Employed, full-time]","Hybrid (some remote, some in-person)","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",14,5,[Data scientist or machine learning specialist...,United Kingdom of Great Britain and Northern I...,32000.0,"[C#, C++, HTML/CSS, JavaScript, Python]","[C#, C++, HTML/CSS, JavaScript, TypeScript]",...,[],[],"[Notepad++, Visual Studio]","[Notepad++, Visual Studio]",[Windows],[Windows],[Git],[Code editor],[Man],NaN
3,"[Employed, full-time]",Fully remote,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",20,17,"[Developer, full-stack]",Israel,60000.0,"[C#, JavaScript, SQL, TypeScript]","[C#, SQL, TypeScript]",...,[],[],"[Notepad++, Visual Studio, Visual Studio Code]","[Notepad++, Visual Studio, Visual Studio Code]",[Windows],[Windows],[Git],"[Code editor, Command-line, Version control ho...",[Man],NaN
4,"[Employed, full-time]","Hybrid (some remote, some in-person)","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",8,3,"[Developer, front-end, Developer, full-stack, ...",United States of America,NaN,"[C#, HTML/CSS, JavaScript, SQL, Swift, TypeScr...","[C#, Elixir, F#, Go, JavaScript, Rust, TypeScr...",...,[npm],"[Docker, Kubernetes]","[Notepad++, Visual Studio, Visual Studio Code,...","[Rider, Visual Studio, Visual Studio Code]",[Windows],"[macOS, Windows]","[Git, Other (please specify):]",[Code editor],[],NaN


In [5]:
X = df.copy().drop("DevType", axis=1)
Y = df["DevType"]

### Onehot Enconding

In [7]:
binarizer = MultiLabelBinarizer()

In [26]:
encoded_dfs = {}
for col in TECH_COLS:
    col_encoded = pd.DataFrame(binarizer.fit_transform(X[col]),
                         columns = binarizer.classes_,
                         index = X[col].index)
    encoded_dfs[col]=col_encoded
    
encoded_dfs

{'LanguageHaveWorkedWith':        APL  Assembly  Bash/Shell  C  C#  C++  COBOL  Clojure  Crystal  Dart  \
 0        0         0           0  0   0    0      0        0        0     0   
 1        0         0           0  0   0    0      0        0        0     0   
 2        0         0           0  0   1    1      0        0        0     0   
 3        0         0           0  0   1    0      0        0        0     0   
 4        0         0           0  0   1    0      0        0        0     0   
 ...    ...       ...         ... ..  ..  ...    ...      ...      ...   ...   
 73263    0         0           1  0   0    0      0        0        0     1   
 73264    0         0           1  0   0    0      0        0        0     0   
 73265    0         0           0  0   0    0      0        0        0     0   
 73266    0         0           0  0   1    0      0        0        0     0   
 73267    0         0           0  0   1    0      0        0        0     0   
 
        ... 

In [27]:
# Merge 1-hot encoded 
X_encoded = pd.concat(encoded_dfs, axis=1)
X_encoded

LanguageHaveWorkedWith                                              \
                         APL Assembly Bash/Shell  C C# C++ COBOL Clojure   
0                          0        0          0  0  0   0     0       0   
1                          0        0          0  0  0   0     0       0   
2                          0        0          0  0  1   1     0       0   
3                          0        0          0  0  1   0     0       0   
4                          0        0          0  0  1   0     0       0   
...                      ...      ...        ... .. ..  ..   ...     ...   
73263                      0        0          1  0  0   0     0       0   
73264                      0        0          1  0  0   0     0       0   
73265                      0        0          0  0  0   0     0       0   
73266                      0        0          0  0  1   0     0       0   
73267                      0        0          0  0  1   0     0       0   

                    ... NEWCollabToolsHaveWorkedWith                  \
      Crystal Dart  ...                        Rider RubyMine Spyder   
0           0    0  ...                            0        0      0   
1           0    0  ...                            0        0      0   
2           0    0  ...                            0        0      0   
3           0    0  ...                            0        0      0   
4           0    0  ...                            0        0      0   
...       ...  ...  ...                          ...      ...    ...   
73263       0    1  ...                            0        0      0   
73264       0    0  ...                            0        0      1   
73265       0    0  ...                            0        0      1   
73266       0    0  ...                            0        0      0   
73267       0    0  ...                            0        0      0   

                                                                           \
      Sublime Text TextMate Vim Visual Studio Visual Studio Code Webstorm   
0                0        0   0             0                  0        0   
1                0        0   0             0                  0        0   
2                0        0   0             1                  0        0   
3                0        0   0             1                  1        0   
4                0        0   0             1                  1        0   
...            ...      ...  ..           ...                ...      ...   
73263            1        0   1             0                  1        0   
73264            0        0   1             0                  1        0   
73265            0        0   0             1                  1        0   
73266            0        0   0             1                  0        0   
73267            0        0   0             1                  1        0   

             
      Xcode  
0         0  
1         0  
2         0  
3         0  
4         1  
...     ...  
73263     0  
73264     0  
73265     0  
73266     0  
73267     0  

[73268 rows x 161 columns]

In [10]:
Y_encoded = pd.DataFrame(binarizer.fit_transform(Y),
                        columns=binarizer.classes_,
                        index=Y.index)
Y_encoded

,Academic researcher,Blockchain,Cloud infrastructure engineer,Data or business analyst,Data scientist or machine learning specialist,Database administrator,Designer,DevOps specialist,"Developer, QA or test","Developer, back-end",...,Engineering manager,Marketing or sales professional,Other (please specify):,Product manager,Project manager,Scientist,Security professional,"Senior Executive (C-Suite, VP, etc.)",Student,System administrator
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73263,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
73264,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
73265,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
73266,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


_________

## Display jobs frequency

In [19]:
jobs_freq = Y_encoded.copy().sum().sort_values(ascending=False).reset_index()

In [20]:
jobs_freq

,index,0
0,"Developer, full-stack",28701
1,"Developer, back-end",26595
2,"Developer, front-end",15915
3,"Developer, desktop or enterprise applications",9546
4,"Developer, mobile",7634
5,DevOps specialist,6170
6,Student,5595
7,Cloud infrastructure engineer,5283
8,Database administrator,4934
9,System administrator,4908


## Display skills frequency 

In [30]:
skills_freq = X_encoded.copy().sum().sort_values(ascending=False).reset_index()
skills_freq.columns = ['group','skill', 'freq']
skills_freq

,group,skill,freq
0,NEWCollabToolsHaveWorkedWith,Visual Studio Code,52523
1,LanguageHaveWorkedWith,JavaScript,46443
2,LanguageHaveWorkedWith,HTML/CSS,39142
3,ToolsTechHaveWorkedWith,npm,35778
4,LanguageHaveWorkedWith,SQL,35127
...,...,...,...
156,ToolsTechHaveWorkedWith,Flow,444
157,LanguageHaveWorkedWith,SAS,435
158,LanguageHaveWorkedWith,OCaml,422
159,LanguageHaveWorkedWith,Crystal,340


In [37]:
fig = px.treemap(skills_freq, 
                 path=['group', 'skill'], 
                 color_continuous_scale='deep',
                 values='freq',color='freq')

fig.update_layout(width=1400, height=700)
fig.show()

fig.write_html(os.path.join(FIG_DIR, 'treemap_skills_freq.html'))


## Create Jobs & Skills Heatmap

In [68]:
df_encoded = pd.concat([X_encoded.droplevel(level=0, axis=1), Y_encoded], axis=1) 
df_encoded.head(3)

,APL,Assembly,Bash/Shell,C,C#,C++,COBOL,Clojure,Crystal,Dart,...,Engineering manager,Marketing or sales professional,Other (please specify):,Product manager,Project manager,Scientist,Security professional,"Senior Executive (C-Suite, VP, etc.)",Student,System administrator
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
roles = Y_encoded.columns
skills = X_encoded.droplevel(level=0, axis=1).columns

In [101]:
roles_skills = []
for role in roles:
    role_mask = df_encoded[role]==1

    role_skills_prcntg = pd.DataFrame({skill: df_encoded.loc[role_mask, skill].mean()*100  for skill in skills},
                                     columns=skills,
                                     index=[role])

    roles_skills.append(role_skills_prcntg)

    
roles_skills = pd.concat(roles_skills)

In [102]:
roles_skills

,APL,Assembly,Bash/Shell,C,C#,C++,COBOL,Clojure,Crystal,Dart,...,Rider,RubyMine,Spyder,Sublime Text,TextMate,Vim,Visual Studio,Visual Studio Code,Webstorm,Xcode
Academic researcher,3.285345,10.557401,38.796604,33.222591,18.530823,38.501292,1.587302,2.510151,1.402732,5.832410,...,3.137689,1.402732,7.641196,17.460317,1.845700,31.450720,23.255814,61.794020,6.164636,8.416390
Blockchain,3.917051,9.523810,32.411674,25.806452,22.734255,28.187404,2.688172,3.609831,2.304147,11.443932,...,5.453149,3.379416,3.533026,21.274962,2.611367,32.258065,28.341014,74.039939,14.746544,16.589862
Cloud infrastructure engineer,1.438577,5.754306,53.681620,17.092561,29.945107,19.098997,1.116790,2.536438,1.041075,6.549309,...,7.287526,2.536438,2.101079,18.360780,1.684649,38.425137,28.430816,75.979557,10.183608,11.584327
Data or business analyst,2.499219,4.998438,30.834114,15.682599,30.021868,17.588254,2.405498,1.749453,1.749453,5.248360,...,3.905030,1.530772,7.560137,17.088410,1.530772,20.274914,36.051234,64.729772,5.716963,9.559513
Data scientist or machine learning specialist,1.723131,6.250000,37.061916,21.699766,17.348131,29.030374,1.401869,2.570093,1.255841,4.935748,...,2.803738,1.401869,10.221963,18.078271,1.693925,29.351636,23.422897,68.399533,5.403037,8.177570
Database administrator,1.661938,6.546413,37.758411,18.504256,37.636806,20.044589,1.722740,1.803810,1.722740,7.316579,...,6.039724,1.965951,2.796919,18.889339,1.702473,26.976084,40.494528,68.423186,9.039319,11.430888
Designer,2.098831,7.252922,28.719447,21.094580,32.784272,25.292242,1.859724,1.647184,1.487779,9.139214,...,4.622742,1.673751,2.470776,20.138151,1.700319,20.138151,39.187035,72.980871,8.979809,14.612115
DevOps specialist,1.199352,4.813614,54.862237,16.693679,33.403566,18.541329,1.134522,2.106969,1.102107,6.369530,...,7.811994,2.495948,1.782820,16.369530,1.442464,39.432739,32.123177,75.089141,10.097245,11.280389
"Developer, QA or test",2.164083,6.686047,40.083979,20.671835,35.755814,23.320413,1.711886,1.937984,1.744186,7.428941,...,6.104651,2.454780,2.293282,18.346253,1.808786,28.552972,37.112403,71.963824,10.174419,13.630491
"Developer, back-end",0.560256,4.248919,32.487310,16.059410,31.866892,19.296860,0.834743,2.105659,0.650498,5.869524,...,6.388419,2.000376,1.598045,17.507050,0.887385,26.185373,33.374694,71.994736,8.569280,8.960331


In [103]:
fig = go.Figure(data=go.Heatmap(z=roles_skills, x=roles_skills.columns,y=roles_skills.index, colorscale='magma', ygap=1))
fig.update_layout(width=2500, height=700)
fig.show()
fig.write_html(os.path.join(FIG_DIR, 'heatmap.html'))

>**These values represent how common the skill is to the role**

## Normalizing features 

In [106]:
std_roles_skills = StandardScaler().fit_transform(roles_skills)
std_roles_skills = pd.DataFrame(std_roles_skills, columns=roles_skills.columns, index=roles_skills.index)

In [107]:
fig = go.Figure(data=go.Heatmap(z=std_roles_skills, x=roles_skills.columns,y=roles_skills.index, colorscale='magma', ygap=1))
fig.update_layout(width=2500, height=700)
fig.show()
fig.write_html(os.path.join(FIG_DIR, 'normalized_heatmap.html'))

>**The standardization is done over the roles, hence it converted the percentege of each skill for a role into a value the represents how specific is this skill to the role compared to other roles**

## Create Jobs dendrogram

In [108]:
fig = ff.create_dendrogram(roles_skills, labels=roles_skills.index, orientation='left', color_threshold=0)
fig.update_layout(height=700, width=1000, showlegend=False)
fig.show()

## Job profiles

In [109]:
#role = np.random.choice(roles)
role = 'Data scientist or machine learning specialist'

In [110]:
single_role_skills = pd.concat([roles_skills.loc[role], std_roles_skills.loc[role]], axis=1)
single_role_skills.columns = ['percentage', 'specificity']
single_role_skills = single_role_skills.sort_values('percentage')

In [111]:
single_role_skills

,percentage,specificity
Uno Platform,0.992991,-0.541135
Pulumi,1.139019,-0.433427
Play Framework,1.197430,-0.516881
Flow,1.255841,-0.357717
Crystal,1.255841,-0.410675
...,...,...
SQL,57.535047,0.033617
Visual Studio Code,68.399533,-0.403883
Pandas,69.655374,3.445605
NumPy,73.335280,3.285377


In [115]:
threshold = 25

single_role_skills = single_role_skills[single_role_skills['percentage'] > threshold]

fig = px.bar(df, 
             y=single_role_skills.index, 
             x=single_role_skills['percentage'], 
             color=single_role_skills['specificity'], 
             color_continuous_scale='orrd', 
             range_color=[std_roles_skills.values.min(),std_roles_skills.values.max()],
             orientation='h')

fig.update_layout(width=800, height=800, title=role)
fig.show()